# Decentralized Federated Learning (P2P) Analysis

This notebook analyzes experiments run in **decentralized mode** using peer-to-peer gossip protocols.

**Features:**
- P2P network topology analysis
- Gossip propagation patterns
- Mixing matrix effectiveness
- Per-client local model performance
- Network convergence without central server
- Communication efficiency metrics

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

## 2. Load Experiment Data

In [ ]:
# Specify experiment directory
logs_base = Path('logs')

# Auto-detect latest decentralized experiment or specify manually
# Common patterns: metropolis_hastings, max_degree, jaccard, matcha
decentralized_dirs = [d for d in logs_base.glob('*') if d.is_dir() and 
                      any(x in d.name.lower() for x in ['metropolis', 'max_degree', 'jaccard', 'matcha', 'p2p'])]

if decentralized_dirs:
    # Get most recent
    experiment_dir = max(decentralized_dirs, key=lambda p: p.stat().st_mtime)
    # Or use specific subdirectory with timestamp
    subdirs = sorted([d for d in experiment_dir.iterdir() if d.is_dir()], reverse=True)
    if subdirs:
        experiment_dir = subdirs[0]
else:
    # Manually specify
    experiment_dir = logs_base / 'metropolis_hastings_comparison' / '2026-02-05_12-00-00'

print(f"Analyzing experiment: {experiment_dir}")
print(f"Experiment path: {experiment_dir.absolute()}")

# Create plots directory
plots_dir = experiment_dir / 'plots'
plots_dir.mkdir(exist_ok=True)
print(f"Plots will be saved to: {plots_dir}")

In [ ]:
# Load configuration
config_file = experiment_dir / 'config.json'
if config_file.exists():
    with open(config_file, 'r') as f:
        config = json.load(f)
    print("\nExperiment Configuration:")
    for key, value in config.items():
        print(f"  {key}: {value}")
else:
    print("Warning: config.json not found")
    config = {}

In [ ]:
# Load CSV files
per_client_file = experiment_dir / 'per_client_metrics.csv'
per_class_file = experiment_dir / 'per_class_metrics.csv'
gradient_file = experiment_dir / 'gradient_metrics.csv'
propagation_file = experiment_dir / 'propagation_metrics.csv'  # P2P specific!
convergence_file = experiment_dir / 'convergence_metrics.csv'
data_dist_file = experiment_dir / 'data_distribution.csv'
comm_eff_file = experiment_dir / 'communication_efficiency.csv'
round_summary_file = experiment_dir / 'round_summary.csv'

# Load available files
dfs = {}
if per_client_file.exists():
    dfs['per_client'] = pd.read_csv(per_client_file)
    print(f"✓ Loaded per_client_metrics: {len(dfs['per_client'])} records")

if per_class_file.exists():
    dfs['per_class'] = pd.read_csv(per_class_file)
    print(f"✓ Loaded per_class_metrics: {len(dfs['per_class'])} records")

if gradient_file.exists():
    dfs['gradient'] = pd.read_csv(gradient_file)
    print(f"✓ Loaded gradient_metrics: {len(dfs['gradient'])} records")

if propagation_file.exists():
    dfs['propagation'] = pd.read_csv(propagation_file)
    print(f"✓ Loaded propagation_metrics: {len(dfs['propagation'])} records")

if convergence_file.exists():
    dfs['convergence'] = pd.read_csv(convergence_file)
    print(f"✓ Loaded convergence_metrics: {len(dfs['convergence'])} records")

if data_dist_file.exists():
    dfs['data_dist'] = pd.read_csv(data_dist_file)
    print(f"✓ Loaded data_distribution: {len(dfs['data_dist'])} records")

if comm_eff_file.exists():
    dfs['comm_eff'] = pd.read_csv(comm_eff_file)
    print(f"✓ Loaded communication_efficiency: {len(dfs['comm_eff'])} records")

if round_summary_file.exists():
    dfs['round_summary'] = pd.read_csv(round_summary_file)
    print(f"✓ Loaded round_summary: {len(dfs['round_summary'])} records")

print(f"\nTotal files loaded: {len(dfs)}")

## 3. Network-Wide Performance

In [ ]:
if 'round_summary' in dfs:
    df_summary = dfs['round_summary']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Average accuracy across network
    axes[0].plot(df_summary['round'], df_summary['avg_test_accuracy'], 
                marker='o', linewidth=2, label='Average Accuracy', color='blue')
    axes[0].fill_between(df_summary['round'], 
                          df_summary['min_test_accuracy'], 
                          df_summary['max_test_accuracy'], 
                          alpha=0.2, label='Min-Max Range')
    axes[0].set_xlabel('Round')
    axes[0].set_ylabel('Test Accuracy (%)')
    axes[0].set_title('Network-Wide Test Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Average loss across network
    axes[1].plot(df_summary['round'], df_summary['avg_test_loss'], 
                marker='o', linewidth=2, label='Average Loss', color='red')
    axes[1].fill_between(df_summary['round'], 
                          df_summary['min_test_loss'], 
                          df_summary['max_test_loss'], 
                          alpha=0.2, label='Min-Max Range')
    axes[1].set_xlabel('Round')
    axes[1].set_ylabel('Test Loss')
    axes[1].set_title('Network-Wide Test Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'network_performance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nFinal Network Performance:")
    print(f"  Average Test Accuracy: {df_summary['avg_test_accuracy'].iloc[-1]:.2f}%")
    print(f"  Average Test Loss: {df_summary['avg_test_loss'].iloc[-1]:.4f}")
    print(f"  Best Average Accuracy: {df_summary['avg_test_accuracy'].max():.2f}% (Round {df_summary.loc[df_summary['avg_test_accuracy'].idxmax(), 'round']:.0f})")
    print(f"  Accuracy Range: [{df_summary['min_test_accuracy'].iloc[-1]:.2f}%, {df_summary['max_test_accuracy'].iloc[-1]:.2f}%]")
else:
    print("Round summary not found")

## 4. P2P Gossip Propagation Analysis

In [ ]:
if 'propagation' in dfs:
    df_prop = dfs['propagation']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Messages sent per round
    msg_per_round = df_prop.groupby('round')['messages_sent'].sum()
    axes[0, 0].bar(msg_per_round.index, msg_per_round.values, alpha=0.7, color='steelblue')
    axes[0, 0].set_xlabel('Round')
    axes[0, 0].set_ylabel('Total Messages Sent')
    axes[0, 0].set_title('Gossip Messages per Round')
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    
    # 2. Messages per client
    final_round = df_prop['round'].max()
    final_prop = df_prop[df_prop['round'] == final_round]
    axes[0, 1].bar(final_prop['client_id'], final_prop['messages_sent'], 
                   alpha=0.7, color='coral')
    axes[0, 1].axhline(final_prop['messages_sent'].mean(), 
                      color='red', linestyle='--', linewidth=2, 
                      label=f"Mean: {final_prop['messages_sent'].mean():.1f}")
    axes[0, 1].set_xlabel('Client ID')
    axes[0, 1].set_ylabel('Messages Sent')
    axes[0, 1].set_title(f'Messages Sent by Client (Round {final_round})')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # 3. Gossip rounds per communication round
    gossip_rounds = df_prop.groupby('round')['gossip_rounds'].first()
    axes[1, 0].plot(gossip_rounds.index, gossip_rounds.values, 
                   marker='o', linewidth=2, color='green')
    axes[1, 0].set_xlabel('Round')
    axes[1, 0].set_ylabel('Gossip Rounds')
    axes[1, 0].set_title('Gossip Iterations per Round')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Active neighbors over rounds
    for client_id in df_prop['client_id'].unique():
        client_prop = df_prop[df_prop['client_id'] == client_id]
        axes[1, 1].plot(client_prop['round'], client_prop['active_neighbors'], 
                       marker='o', alpha=0.6, label=f'Client {client_id}')
    axes[1, 1].set_xlabel('Round')
    axes[1, 1].set_ylabel('Active Neighbors')
    axes[1, 1].set_title('Active Neighbor Connections')
    axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'propagation_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nGossip Propagation Statistics:")
    print(f"  Total Messages Sent: {df_prop['messages_sent'].sum()}")
    print(f"  Average Messages per Round: {msg_per_round.mean():.1f}")
    print(f"  Average Active Neighbors: {df_prop['active_neighbors'].mean():.2f}")
    print(f"  Average Gossip Rounds: {gossip_rounds.mean():.2f}")
else:
    print("Propagation metrics not found")

## 5. Mixing Matrix and Consensus

In [ ]:
if 'propagation' in dfs:
    df_prop = dfs['propagation']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Consensus error evolution
    consensus_per_round = df_prop.groupby('round')['consensus_error'].mean()
    axes[0, 0].semilogy(consensus_per_round.index, consensus_per_round.values, 
                        marker='o', linewidth=2, color='purple')
    axes[0, 0].set_xlabel('Round')
    axes[0, 0].set_ylabel('Consensus Error (log scale)')
    axes[0, 0].set_title('Network Consensus Error Evolution')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Model divergence
    divergence_per_round = df_prop.groupby('round')['model_divergence'].mean()
    axes[0, 1].plot(divergence_per_round.index, divergence_per_round.values, 
                   marker='o', linewidth=2, color='red')
    axes[0, 1].set_xlabel('Round')
    axes[0, 1].set_ylabel('Model Divergence')
    axes[0, 1].set_title('Average Model Divergence')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Spectral gap (mixing matrix quality indicator)
    spectral_gap = df_prop.groupby('round')['spectral_gap'].first()
    axes[1, 0].plot(spectral_gap.index, spectral_gap.values, 
                   marker='o', linewidth=2, color='teal')
    axes[1, 0].axhline(spectral_gap.mean(), color='red', linestyle='--', 
                      linewidth=2, label=f'Mean: {spectral_gap.mean():.4f}')
    axes[1, 0].set_xlabel('Round')
    axes[1, 0].set_ylabel('Spectral Gap')
    axes[1, 0].set_title('Mixing Matrix Spectral Gap (Higher = Better)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Consensus error per client (final round)
    final_round = df_prop['round'].max()
    final_consensus = df_prop[df_prop['round'] == final_round]
    axes[1, 1].bar(final_consensus['client_id'], final_consensus['consensus_error'], 
                   alpha=0.7, color='orange')
    axes[1, 1].set_xlabel('Client ID')
    axes[1, 1].set_ylabel('Consensus Error')
    axes[1, 1].set_title(f'Consensus Error by Client (Round {final_round})')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'consensus_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nConsensus Statistics:")
    print(f"  Final Consensus Error: {consensus_per_round.iloc[-1]:.6f}")
    print(f"  Final Model Divergence: {divergence_per_round.iloc[-1]:.6f}")
    print(f"  Average Spectral Gap: {spectral_gap.mean():.4f}")
    print(f"  Convergence Rate: {(consensus_per_round.iloc[0] - consensus_per_round.iloc[-1]) / len(consensus_per_round):.6f} per round")
else:
    print("Propagation metrics not found")

## 6. Per-Client Local Model Performance

In [ ]:
if 'per_client' in dfs:
    df_client = dfs['per_client']
    
    # Get final round data
    final_round = df_client['round'].max()
    final_data = df_client[df_client['round'] == final_round]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Client accuracy evolution
    for client_id in df_client['client_id'].unique():
        client_data = df_client[df_client['client_id'] == client_id]
        # Get last epoch of each round
        last_epochs = client_data.groupby('round').last()
        axes[0, 0].plot(last_epochs.index, last_epochs['test_accuracy'], 
                       marker='o', alpha=0.7, label=f'Client {client_id}')
    axes[0, 0].set_xlabel('Round')
    axes[0, 0].set_ylabel('Test Accuracy (%)')
    axes[0, 0].set_title('Per-Client Test Accuracy Evolution')
    axes[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Client loss evolution
    for client_id in df_client['client_id'].unique():
        client_data = df_client[df_client['client_id'] == client_id]
        last_epochs = client_data.groupby('round').last()
        axes[0, 1].plot(last_epochs.index, last_epochs['test_loss'], 
                       marker='o', alpha=0.7, label=f'Client {client_id}')
    axes[0, 1].set_xlabel('Round')
    axes[0, 1].set_ylabel('Test Loss')
    axes[0, 1].set_title('Per-Client Test Loss Evolution')
    axes[0, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Final round accuracy comparison
    final_acc = final_data.groupby('client_id')['test_accuracy'].last()
    axes[1, 0].bar(final_acc.index, final_acc.values, alpha=0.7, color='steelblue')
    axes[1, 0].axhline(final_acc.mean(), color='red', linestyle='--', 
                      linewidth=2, label=f'Mean: {final_acc.mean():.2f}%')
    axes[1, 0].set_xlabel('Client ID')
    axes[1, 0].set_ylabel('Test Accuracy (%)')
    axes[1, 0].set_title(f'Final Test Accuracy by Client (Round {final_round})')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # 4. Accuracy distribution and fairness
    axes[1, 1].hist(final_acc.values, bins=15, alpha=0.7, color='green', edgecolor='black')
    axes[1, 1].axvline(final_acc.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
    axes[1, 1].axvline(final_acc.median(), color='blue', linestyle='--', linewidth=2, label='Median')
    axes[1, 1].set_xlabel('Test Accuracy (%)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Client Accuracy Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'per_client_local_performance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nClient Local Model Statistics (Final Round):")
    print(f"  Mean Accuracy: {final_acc.mean():.2f}%")
    print(f"  Std Dev: {final_acc.std():.2f}%")
    print(f"  Min Accuracy: {final_acc.min():.2f}% (Client {final_acc.idxmin()})")
    print(f"  Max Accuracy: {final_acc.max():.2f}% (Client {final_acc.idxmax()})")
    print(f"  Fairness Gap: {final_acc.max() - final_acc.min():.2f}%")
else:
    print("Per-client metrics not found")

## 7. Communication Efficiency

In [ ]:
if 'comm_eff' in dfs:
    df_comm = dfs['comm_eff']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Bytes sent per round
    bytes_per_round = df_comm.groupby('round')['bytes_sent'].sum()
    axes[0, 0].plot(bytes_per_round.index, bytes_per_round.values / 1e6, 
                   marker='o', linewidth=2, color='blue')
    axes[0, 0].set_xlabel('Round')
    axes[0, 0].set_ylabel('Total Bytes Sent (MB)')
    axes[0, 0].set_title('Communication Volume per Round')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Bytes received per round
    bytes_recv_per_round = df_comm.groupby('round')['bytes_received'].sum()
    axes[0, 1].plot(bytes_recv_per_round.index, bytes_recv_per_round.values / 1e6, 
                   marker='o', linewidth=2, color='green')
    axes[0, 1].set_xlabel('Round')
    axes[0, 1].set_ylabel('Total Bytes Received (MB)')
    axes[0, 1].set_title('Data Reception per Round')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Communication overhead per client
    final_round = df_comm['round'].max()
    final_comm = df_comm[df_comm['round'] == final_round]
    axes[1, 0].bar(final_comm['client_id'], final_comm['bytes_sent'] / 1e6, 
                   alpha=0.7, color='coral', label='Sent')
    axes[1, 0].bar(final_comm['client_id'], final_comm['bytes_received'] / 1e6, 
                   alpha=0.7, color='lightgreen', label='Received')
    axes[1, 0].set_xlabel('Client ID')
    axes[1, 0].set_ylabel('Data (MB)')
    axes[1, 0].set_title(f'Communication Volume by Client (Round {final_round})')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # 4. Communication rounds
    comm_rounds = df_comm.groupby('round')['communication_rounds'].first()
    axes[1, 1].plot(comm_rounds.index, comm_rounds.values, 
                   marker='o', linewidth=2, color='purple')
    axes[1, 1].set_xlabel('Round')
    axes[1, 1].set_ylabel('Communication Rounds')
    axes[1, 1].set_title('Communication Rounds per Training Round')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'communication_efficiency.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nCommunication Efficiency:")
    print(f"  Total Data Sent: {df_comm['bytes_sent'].sum() / 1e6:.2f} MB")
    print(f"  Total Data Received: {df_comm['bytes_received'].sum() / 1e6:.2f} MB")
    print(f"  Average per Round: {bytes_per_round.mean() / 1e6:.2f} MB sent")
    print(f"  Average Communication Rounds: {comm_rounds.mean():.2f}")
else:
    print("Communication efficiency metrics not found")

## 8. Convergence and Fairness

In [ ]:
if 'convergence' in dfs:
    df_conv = dfs['convergence']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Convergence rate
    axes[0, 0].plot(df_conv['round'], df_conv['accuracy_convergence_rate'], 
                   marker='o', linewidth=2, color='green')
    axes[0, 0].axhline(0, color='black', linestyle='--', linewidth=1)
    axes[0, 0].set_xlabel('Round')
    axes[0, 0].set_ylabel('Convergence Rate')
    axes[0, 0].set_title('Accuracy Convergence Rate')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Client accuracy spread
    axes[0, 1].fill_between(df_conv['round'], 
                            df_conv['client_accuracy_min'], 
                            df_conv['client_accuracy_max'], 
                            alpha=0.3, label='Min-Max Range')
    axes[0, 1].plot(df_conv['round'], df_conv['client_accuracy_mean'], 
                   marker='o', linewidth=2, color='blue', label='Mean')
    axes[0, 1].set_xlabel('Round')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_title('Client Accuracy Spread Over Rounds')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Standard deviation (fairness indicator)
    axes[1, 0].plot(df_conv['round'], df_conv['client_accuracy_std'], 
                   marker='o', linewidth=2, color='purple')
    axes[1, 0].set_xlabel('Round')
    axes[1, 0].set_ylabel('Standard Deviation (%)')
    axes[1, 0].set_title('Client Accuracy Std Dev (Fairness Indicator)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Stragglers detection
    straggler_counts = df_conv.groupby('round')['stragglers_count'].first()
    axes[1, 1].plot(straggler_counts.index, straggler_counts.values, 
                   marker='o', linewidth=2, color='red')
    axes[1, 1].fill_between(straggler_counts.index, straggler_counts.values, 
                            alpha=0.3, color='red')
    axes[1, 1].set_xlabel('Round')
    axes[1, 1].set_ylabel('Number of Stragglers')
    axes[1, 1].set_title('Straggler Detection Over Rounds')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'convergence_fairness.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nConvergence and Fairness:")
    print(f"  Average Convergence Rate: {df_conv['accuracy_convergence_rate'].mean():.6f}")
    print(f"  Final Fairness (Std Dev): {df_conv['client_accuracy_std'].iloc[-1]:.2f}%")
    print(f"  Total Stragglers: {df_conv['stragglers_count'].sum():.0f}")
else:
    print("Convergence metrics not found")

## 9. Data Distribution Heterogeneity

In [ ]:
if 'data_dist' in dfs:
    df_dist = dfs['data_dist']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 1. Heterogeneity scores
    axes[0].bar(df_dist['client_id'], df_dist['data_heterogeneity_score'], 
               alpha=0.7, color='purple')
    axes[0].axhline(df_dist['data_heterogeneity_score'].mean(), 
                   color='red', linestyle='--', linewidth=2, 
                   label=f"Mean: {df_dist['data_heterogeneity_score'].mean():.4f}")
    axes[0].set_xlabel('Client ID')
    axes[0].set_ylabel('Heterogeneity Score (KL Divergence)')
    axes[0].set_title('Data Heterogeneity by Client')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # 2. Sample distribution
    axes[1].bar(df_dist['client_id'], df_dist['total_samples'], 
               alpha=0.7, color='teal')
    axes[1].axhline(df_dist['total_samples'].mean(), 
                   color='red', linestyle='--', linewidth=2, 
                   label=f"Mean: {df_dist['total_samples'].mean():.0f}")
    axes[1].set_xlabel('Client ID')
    axes[1].set_ylabel('Total Samples')
    axes[1].set_title('Dataset Size by Client')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'data_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\nData Distribution Statistics:")
    print(f"  Average Heterogeneity: {df_dist['data_heterogeneity_score'].mean():.4f}")
    print(f"  Total Samples: {df_dist['total_samples'].sum()}")
    print(f"  Imbalance Ratio: {df_dist['total_samples'].max() / df_dist['total_samples'].min():.2f}x")
else:
    print("Data distribution metrics not found")

## 10. Summary Report

In [ ]:
print("=" * 80)
print("DECENTRALIZED FEDERATED LEARNING (P2P) - EXPERIMENT SUMMARY")
print("=" * 80)

print("\n📋 CONFIGURATION")
print("-" * 80)
if config:
    print(f"  Mode: {config.get('type', 'decentralized')}")
    print(f"  Mixing Method: {config.get('mixing_method', 'N/A')}")
    print(f"  Clients: {config.get('num_clients', 'N/A')}")
    print(f"  Rounds: {config.get('rounds', 'N/A')}")
    print(f"  Epochs: {config.get('epochs', 'N/A')}")
    print(f"  Dataset: {config.get('dataset', 'N/A')}")
    print(f"  Model: {config.get('model', 'N/A')}")
    print(f"  Partition: {config.get('partition', 'N/A')}")

if 'round_summary' in dfs:
    df_summary = dfs['round_summary']
    print("\n📊 NETWORK PERFORMANCE")
    print("-" * 80)
    print(f"  Average Test Accuracy: {df_summary['avg_test_accuracy'].iloc[-1]:.2f}%")
    print(f"  Average Test Loss: {df_summary['avg_test_loss'].iloc[-1]:.4f}")
    print(f"  Best Average Accuracy: {df_summary['avg_test_accuracy'].max():.2f}%")
    print(f"  Accuracy Range: [{df_summary['min_test_accuracy'].iloc[-1]:.2f}%, {df_summary['max_test_accuracy'].iloc[-1]:.2f}%]")

if 'propagation' in dfs:
    df_prop = dfs['propagation']
    print("\n🔄 GOSSIP PROPAGATION")
    print("-" * 80)
    print(f"  Total Messages: {df_prop['messages_sent'].sum():.0f}")
    print(f"  Avg Messages per Round: {df_prop.groupby('round')['messages_sent'].sum().mean():.1f}")
    print(f"  Avg Active Neighbors: {df_prop['active_neighbors'].mean():.2f}")
    print(f"  Final Consensus Error: {df_prop.groupby('round')['consensus_error'].mean().iloc[-1]:.6f}")
    print(f"  Avg Spectral Gap: {df_prop.groupby('round')['spectral_gap'].first().mean():.4f}")

if 'comm_eff' in dfs:
    df_comm = dfs['comm_eff']
    print("\n📡 COMMUNICATION")
    print("-" * 80)
    print(f"  Total Data Sent: {df_comm['bytes_sent'].sum() / 1e6:.2f} MB")
    print(f"  Total Data Received: {df_comm['bytes_received'].sum() / 1e6:.2f} MB")
    print(f"  Avg per Round: {df_comm.groupby('round')['bytes_sent'].sum().mean() / 1e6:.2f} MB")

if 'convergence' in dfs:
    df_conv = dfs['convergence']
    print("\n🎯 CONVERGENCE & FAIRNESS")
    print("-" * 80)
    print(f"  Total Stragglers: {df_conv['stragglers_count'].sum():.0f}")
    print(f"  Avg Convergence Rate: {df_conv['accuracy_convergence_rate'].mean():.6f}")
    print(f"  Final Fairness (Std): {df_conv['client_accuracy_std'].iloc[-1]:.2f}%")

if 'data_dist' in dfs:
    df_dist = dfs['data_dist']
    print("\n📊 DATA DISTRIBUTION")
    print("-" * 80)
    print(f"  Avg Heterogeneity: {df_dist['data_heterogeneity_score'].mean():.4f}")
    print(f"  Total Samples: {df_dist['total_samples'].sum()}")
    print(f"  Imbalance Ratio: {df_dist['total_samples'].max() / df_dist['total_samples'].min():.2f}x")

print("\n" + "=" * 80)
print("Analysis complete! All plots saved to:", plots_dir)
print("=" * 80)